In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, make_blobs
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 10

print("=" * 70)
print("ANÁLISIS COMPLETO DE PCA - IMPLEMENTACIÓN DESDE CERO")
print("=" * 70)
print("Librerías cargadas exitosamente")


In [ ]:
class Matrix:
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.data = np.zeros((rows, cols))
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def copy(self):
        new_matrix = Matrix(self.rows, self.cols)
        new_matrix.data = self.data.copy()
        return new_matrix
    
    def transpose(self):
        new_matrix = Matrix(self.cols, self.rows)
        new_matrix.data = self.data.T
        return new_matrix
    
    def multiply(self, other):
        if self.cols != other.rows:
            raise ValueError("Dimensiones incompatibles para multiplicación")
        result = Matrix(self.rows, other.cols)
        result.data = np.dot(self.data, other.data)
        return result
    
    def print_matrix(self, name="Matrix", max_rows=5, max_cols=5):
        print(f"\n{name} ({self.rows} x {self.cols}):")
        rows_to_show = min(max_rows, self.rows)
        cols_to_show = min(max_cols, self.cols)
        
        for i in range(rows_to_show):
            for j in range(cols_to_show):
                print(f"{self.data[i, j]:10.6f}", end=" ")
            if self.cols > cols_to_show:
                print("...", end="")
            print()
        if self.rows > rows_to_show:
            print("...")
        print()

print("✓ Clase Matrix implementada")


In [ ]:
def generate_synthetic_data(n_samples=500, n_features=10, n_informative=8, 
                           n_redundant=2, random_state=42):
    print(f"Generando datos sintéticos...")
    print(f"  - Muestras (N): {n_samples}")
    print(f"  - Dimensiones (M): {n_features}")
    print(f"  - Características informativas: {n_informative}")
    print(f"  - Características redundantes: {n_redundant}")
    
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=n_informative,
        n_redundant=n_redundant,
        n_clusters_per_class=2,
        random_state=random_state,
        shuffle=True
    )
    
    X = X * np.random.uniform(0.5, 2.0, size=n_features)
    X = X + np.random.uniform(-5, 5, size=n_features)
    
    print(f"  - Shape de los datos: {X.shape}")
    print(f"  - Media de cada dimensión: {X.mean(axis=0)[:3]}... (primeras 3)")
    print(f"  - Desviación estándar: {X.std(axis=0)[:3]}... (primeras 3)")
    
    return X, y

def generate_blobs_data(n_samples=500, n_features=10, centers=3, random_state=42):
    print(f"Generando datos de clusters...")
    print(f"  - Muestras (N): {n_samples}")
    print(f"  - Dimensiones (M): {n_features}")
    print(f"  - Número de clusters: {centers}")
    
    X, y = make_blobs(
        n_samples=n_samples,
        n_features=n_features,
        centers=centers,
        cluster_std=2.0,
        random_state=random_state
    )
    
    print(f"  - Shape de los datos: {X.shape}")
    return X, y

print("✓ Funciones de generación de datos implementadas")


In [ ]:
def compute_mean(matrix):
    if isinstance(matrix, Matrix):
        data = matrix.data
    else:
        data = matrix
    
    mean = np.mean(data, axis=0)
    return mean

def center_data(matrix, mean):
    if isinstance(matrix, Matrix):
        matrix.data = matrix.data - mean
    else:
        matrix = matrix - mean
    return matrix

def compute_covariance(matrix):
    if isinstance(matrix, Matrix):
        data = matrix.data
    else:
        data = matrix
    
    n = data.shape[0]
    cov = np.dot(data.T, data) / (n - 1)
    return cov

def vector_norm(vec):
    return np.sqrt(np.sum(vec**2))

def vector_normalize(vec):
    norm = vector_norm(vec)
    if norm > 1e-10:
        return vec / norm
    return vec

def vector_dot(vec1, vec2):
    return np.dot(vec1, vec2)

print("✓ Funciones estadísticas básicas implementadas")


In [ ]:
def compute_eigen(cov_matrix, max_iterations=1000, tolerance=1e-10):
    n = cov_matrix.shape[0]
    A = cov_matrix.copy()
    eigenvalues = np.zeros(n)
    eigenvectors = np.zeros((n, n))
    
    for k in range(n):
        v = np.ones(n) / np.sqrt(n)
        lambda_val = 0.0
        
        for iter in range(max_iterations):
            v_new = np.dot(A, v)
            lambda_new = np.dot(v_new, v)
            v_new = vector_normalize(v_new)
            
            if abs(lambda_new - lambda_val) < tolerance:
                lambda_val = lambda_new
                v = v_new
                break
            
            lambda_val = lambda_new
            v = v_new
        
        eigenvalues[k] = lambda_val
        eigenvectors[:, k] = v
        
        A = A - lambda_val * np.outer(v, v)
    
    return eigenvalues, eigenvectors

def sort_eigen(eigenvalues, eigenvectors):
    n = len(eigenvalues)
    for i in range(n - 1):
        for j in range(n - i - 1):
            if eigenvalues[j] < eigenvalues[j + 1]:
                eigenvalues[j], eigenvalues[j + 1] = eigenvalues[j + 1], eigenvalues[j]
                eigenvectors[:, j], eigenvectors[:, j + 1] = eigenvectors[:, j + 1], eigenvectors[:, j].copy()

print("✓ Algoritmo de eigenvalores implementado")


In [ ]:
class PCAModel:
    def __init__(self, n_components):
        self.n_components = n_components
        self.mean = None
        self.eigenvalues = None
        self.eigenvectors = None
        self.explained_variance_ratio = 0.0

def pca_fit(data, n_components):
    print("\n" + "=" * 50)
    print("ENTRENANDO MODELO PCA")
    print("=" * 50)
    print(f"Input shape: {data.shape[0]} muestras x {data.shape[1]} características")
    print(f"Componentes objetivo: {n_components}")
    
    model = PCAModel(n_components)
    
    mean = compute_mean(data)
    model.mean = mean
    
    data_centered = center_data(data.copy(), mean)
    
    cov_matrix = compute_covariance(data_centered)
    print(f"Covariance matrix: {cov_matrix.shape[0]} x {cov_matrix.shape[1]}")
    
    eigenvalues, eigenvectors = compute_eigen(cov_matrix)
    sort_eigen(eigenvalues, eigenvectors)
    
    model.eigenvalues = eigenvalues
    model.eigenvectors = eigenvectors
    
    total_variance = np.sum(eigenvalues)
    explained_variance = np.sum(eigenvalues[:n_components])
    model.explained_variance_ratio = explained_variance / total_variance
    
    print(f"\nVarianza explicada: {model.explained_variance_ratio:.4f} ({model.explained_variance_ratio*100:.2f}%)")
    print(f"\nTop eigenvalues:")
    for i in range(min(n_components, 5)):
        print(f"  PC{i+1}: {eigenvalues[i]:.6f}")
    
    return model

def pca_transform(model, data):
    data_centered = center_data(data.copy(), model.mean)
    components = model.eigenvectors[:, :model.n_components]
    projected = np.dot(data_centered, components)
    return projected

print("✓ Clase PCAModel y funciones PCA implementadas")


In [ ]:
print("\n" + "=" * 70)
print("PASO 1: GENERACIÓN DE DATOS SINTÉTICOS")
print("=" * 70)

n_samples = 500
n_features = 10
n_components = 2

X, y = generate_synthetic_data(
    n_samples=n_samples,
    n_features=n_features,
    n_informative=max(2, n_features - 2),
    n_redundant=min(2, n_features // 2),
    random_state=42
)

print(f"\n✓ Datos generados exitosamente")
print(f"  - Shape: {X.shape}")
print(f"  - Clases únicas: {len(np.unique(y))}")
print(f"  - Distribución de clases: {np.bincount(y)}")


In [ ]:
print("\n" + "=" * 70)
print("PASO 2: APLICACIÓN DE PCA - IMPLEMENTACIÓN PROPIA")
print("=" * 70)

pca_model = pca_fit(X, n_components)
X_pca_custom = pca_transform(pca_model, X)

print(f"\n✓ PCA aplicado exitosamente")
print(f"  - Shape original: {X.shape}")
print(f"  - Shape reducido: {X_pca_custom.shape}")
print(f"  - Reducción dimensional: {(1 - n_components/n_features)*100:.1f}%")


In [ ]:
print("\n" + "=" * 70)
print("PASO 3: APLICACIÓN DE PCA - SKLEARN (REFERENCIA)")
print("=" * 70)

pca_sklearn = PCA(n_components=n_components)
X_pca_sklearn = pca_sklearn.fit_transform(X)

print(f"✓ PCA sklearn aplicado")
print(f"  - Shape: {X_pca_sklearn.shape}")
print(f"  - Varianza explicada: {pca_sklearn.explained_variance_ratio_.sum():.4f} ({pca_sklearn.explained_variance_ratio_.sum()*100:.2f}%)")
print(f"  - Componentes principales:")
for i, var in enumerate(pca_sklearn.explained_variance_ratio_):
    print(f"    PC{i+1}: {var*100:.2f}%")


In [ ]:
print("\n" + "=" * 70)
print("PASO 4: COMPARACIÓN NUMÉRICA")
print("=" * 70)

X_pca_custom_adjusted = X_pca_custom.copy()
flipped_components = []

for i in range(X_pca_custom.shape[1]):
    corr = np.corrcoef(X_pca_sklearn[:, i], X_pca_custom[:, i])[0, 1]
    if corr < 0:
        X_pca_custom_adjusted[:, i] = -X_pca_custom[:, i]
        flipped_components.append(i + 1)

mse = mean_squared_error(X_pca_sklearn, X_pca_custom_adjusted)
mae = mean_absolute_error(X_pca_sklearn, X_pca_custom_adjusted)

correlations = []
for i in range(X_pca_custom.shape[1]):
    corr = np.corrcoef(X_pca_sklearn[:, i], X_pca_custom_adjusted[:, i])[0, 1]
    correlations.append(corr)

diff = X_pca_sklearn - X_pca_custom_adjusted
max_diff = np.max(np.abs(diff))
mean_diff = np.mean(np.abs(diff))

sign_note = f" (componentes invertidos: PC{', PC'.join(map(str, flipped_components))})" if flipped_components else ""

print(f"\nMétricas de comparación{sign_note}:")
print(f"  - MSE (Error Cuadrático Medio):        {mse:.6e}")
print(f"  - MAE (Error Absoluto Medio):          {mae:.6e}")
print(f"  - Diferencia máxima:                   {max_diff:.6e}")
print(f"  - Diferencia media (abs):              {mean_diff:.6e}")
print(f"\nCorrelación por componente:")
for i, corr in enumerate(correlations):
    print(f"  - PC{i+1}: {corr:.6f}")

avg_corr = np.mean(correlations)
if avg_corr > 0.99:
    status = "EXCELENTE ✓"
elif avg_corr > 0.95:
    status = "BUENA ✓"
else:
    status = "REVISAR ⚠"

print(f"\nEstado de la implementación: {status}")
print(f"Correlación promedio: {avg_corr:.6f}")


In [ ]:
print("\n" + "=" * 70)
print("PASO 5: VISUALIZACIONES COMPARATIVAS")
print("=" * 70)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

colors = plt.cm.Set1(np.linspace(0, 1, len(np.unique(y))))

for idx, class_label in enumerate(np.unique(y)):
    mask = y == class_label
    axes[0, 0].scatter(X_pca_custom_adjusted[mask, 0], X_pca_custom_adjusted[mask, 1], 
                       alpha=0.6, s=30, label=f'Clase {int(class_label)}', c=[colors[idx]])
axes[0, 0].set_xlabel('PC1')
axes[0, 0].set_ylabel('PC2')
axes[0, 0].set_title('PCA - Implementación Propia')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

for idx, class_label in enumerate(np.unique(y)):
    mask = y == class_label
    axes[0, 1].scatter(X_pca_sklearn[mask, 0], X_pca_sklearn[mask, 1], 
                       alpha=0.6, s=30, label=f'Clase {int(class_label)}', c=[colors[idx]])
axes[0, 1].set_xlabel('PC1')
axes[0, 1].set_ylabel('PC2')
axes[0, 1].set_title('PCA - sklearn')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].scatter(X_pca_sklearn[:, 0], X_pca_custom_adjusted[:, 0], alpha=0.5, s=20)
min_val = min(X_pca_sklearn[:, 0].min(), X_pca_custom_adjusted[:, 0].min())
max_val = max(X_pca_sklearn[:, 0].max(), X_pca_custom_adjusted[:, 0].max())
axes[1, 0].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect match')
axes[1, 0].set_xlabel('sklearn PC1')
axes[1, 0].set_ylabel('Custom PC1')
axes[1, 0].set_title('Correlación PC1')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_aspect('equal', adjustable='box')

axes[1, 1].scatter(X_pca_sklearn[:, 1], X_pca_custom_adjusted[:, 1], alpha=0.5, s=20)
min_val = min(X_pca_sklearn[:, 1].min(), X_pca_custom_adjusted[:, 1].min())
max_val = max(X_pca_sklearn[:, 1].max(), X_pca_custom_adjusted[:, 1].max())
axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect match')
axes[1, 1].set_xlabel('sklearn PC2')
axes[1, 1].set_ylabel('Custom PC2')
axes[1, 1].set_title('Correlación PC2')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

print("✓ Visualizaciones generadas exitosamente")


In [ ]:
print("\n" + "=" * 70)
print("PASO 6: ANÁLISIS DE CONTORNOS Y SUPERPOSICIÓN")
print("=" * 70)

fig, ax = plt.subplots(figsize=(14, 12))

try:
    from scipy.stats import gaussian_kde
    
    for idx, class_label in enumerate(np.unique(y)):
        mask = y == class_label
        
        if np.sum(mask) > 3:
            xy_sklearn = np.vstack([X_pca_sklearn[mask, 0], X_pca_sklearn[mask, 1]])
            kde_sklearn = gaussian_kde(xy_sklearn)
            
            xy_custom = np.vstack([X_pca_custom_adjusted[mask, 0], X_pca_custom_adjusted[mask, 1]])
            kde_custom = gaussian_kde(xy_custom)
            
            x_min = min(X_pca_sklearn[:, 0].min(), X_pca_custom_adjusted[:, 0].min()) - 2
            x_max = max(X_pca_sklearn[:, 0].max(), X_pca_custom_adjusted[:, 0].max()) + 2
            y_min = min(X_pca_sklearn[:, 1].min(), X_pca_custom_adjusted[:, 1].min()) - 2
            y_max = max(X_pca_sklearn[:, 1].max(), X_pca_custom_adjusted[:, 1].max()) + 2
            
            xx, yy = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]
            positions = np.vstack([xx.ravel(), yy.ravel()])
            
            z_sklearn = np.reshape(kde_sklearn(positions).T, xx.shape)
            z_custom = np.reshape(kde_custom(positions).T, xx.shape)
            
            ax.contour(xx, yy, z_sklearn, levels=5, colors=['blue'], alpha=0.6, linewidths=2, linestyles='solid')
            ax.contour(xx, yy, z_custom, levels=5, colors=['red'], alpha=0.6, linewidths=2, linestyles='dashed')
    
    for idx, class_label in enumerate(np.unique(y)):
        mask = y == class_label
        ax.scatter(X_pca_sklearn[mask, 0], X_pca_sklearn[mask, 1],
                  c=[colors[idx]], alpha=0.5, s=50,
                  edgecolors='blue', linewidths=1.5,
                  marker='o', label=f'sklearn Clase {int(class_label)}')
        
        ax.scatter(X_pca_custom_adjusted[mask, 0], X_pca_custom_adjusted[mask, 1],
                  c=[colors[idx]], alpha=1.0, s=60,
                  edgecolors='red', linewidths=2.0,
                  marker='x', label=f'Custom Clase {int(class_label)}')
    
    ax.set_xlabel('PC1', fontsize=12, fontweight='bold')
    ax.set_ylabel('PC2', fontsize=12, fontweight='bold')
    ax.set_title('Superposición de Contornos\nsklearn (azul sólido) | Custom (rojo punteado)', 
                fontsize=14, fontweight='bold', pad=20)
    ax.grid(True, alpha=0.2, linestyle='--')
    ax.legend(loc='best', framealpha=0.9, fontsize=8, ncol=2)
    
    plt.tight_layout()
    plt.show()
    
    print("✓ Análisis de contornos completado")
    
except ImportError:
    print("⚠ scipy no disponible, saltando análisis de contornos")
    print("  Instalar con: pip install scipy")


In [ ]:
print("\n" + "=" * 70)
print("PASO 7: ANÁLISIS DE DISTRIBUCIONES DE DIFERENCIAS")
print("=" * 70)

fig, axes = plt.subplots(1, n_components, figsize=(8*n_components, 6))

if n_components == 1:
    axes = [axes]

for i in range(n_components):
    diff = X_pca_sklearn[:, i] - X_pca_custom_adjusted[:, i]
    
    axes[i].hist(diff, bins=50, alpha=0.7, edgecolor='black')
    axes[i].axvline(0, color='red', linestyle='--', linewidth=2, label='Cero')
    axes[i].axvline(np.mean(diff), color='green', linestyle='-', 
                   linewidth=2, label=f'Media: {np.mean(diff):.2e}')
    
    axes[i].set_xlabel('Diferencia (sklearn - Custom)')
    axes[i].set_ylabel('Frecuencia')
    axes[i].set_title(f'Distribución de Diferencias - PC{i+1}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Análisis de distribuciones completado")


In [ ]:
print("\n" + "=" * 70)
print("PASO 8: ANÁLISIS CON DIFERENTES TIPOS DE DATOS")
print("=" * 70)

print("Probando con datos de clusters...")
X_blobs, y_blobs = generate_blobs_data(
    n_samples=300,
    n_features=8,
    centers=4,
    random_state=42
)

pca_model_blobs = pca_fit(X_blobs, 2)
X_pca_blobs_custom = pca_transform(pca_model_blobs, X_blobs)

pca_sklearn_blobs = PCA(n_components=2)
X_pca_blobs_sklearn = pca_sklearn_blobs.fit_transform(X_blobs)

X_pca_blobs_custom_adjusted = X_pca_blobs_custom.copy()
for i in range(X_pca_blobs_custom.shape[1]):
    corr = np.corrcoef(X_pca_blobs_sklearn[:, i], X_pca_blobs_custom[:, i])[0, 1]
    if corr < 0:
        X_pca_blobs_custom_adjusted[:, i] = -X_pca_blobs_custom[:, i]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

colors_blobs = plt.cm.Set1(np.linspace(0, 1, len(np.unique(y_blobs))))

for idx, class_label in enumerate(np.unique(y_blobs)):
    mask = y_blobs == class_label
    axes[0].scatter(X_pca_blobs_custom_adjusted[mask, 0], X_pca_blobs_custom_adjusted[mask, 1], 
                   alpha=0.6, s=30, label=f'Cluster {int(class_label)}', c=[colors_blobs[idx]])
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[0].set_title('PCA Custom - Datos de Clusters')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

for idx, class_label in enumerate(np.unique(y_blobs)):
    mask = y_blobs == class_label
    axes[1].scatter(X_pca_blobs_sklearn[mask, 0], X_pca_blobs_sklearn[mask, 1], 
                   alpha=0.6, s=30, label=f'Cluster {int(class_label)}', c=[colors_blobs[idx]])
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')
axes[1].set_title('PCA sklearn - Datos de Clusters')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Análisis con datos de clusters completado")


In [ ]:
print("\n" + "=" * 70)
print("PASO 9: ANÁLISIS DE VARIANZA EXPLICADA")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

eigenvalues_custom = pca_model.eigenvalues
eigenvalues_sklearn = pca_sklearn.explained_variance_

axes[0].bar(range(1, len(eigenvalues_custom) + 1), eigenvalues_custom, alpha=0.7, label='Custom', color='blue')
axes[0].bar(range(1, len(eigenvalues_sklearn) + 1), eigenvalues_sklearn, alpha=0.7, label='sklearn', color='red')
axes[0].set_xlabel('Componente Principal')
axes[0].set_ylabel('Eigenvalue')
axes[0].set_title('Comparación de Eigenvalues')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

explained_var_custom = eigenvalues_custom / np.sum(eigenvalues_custom)
explained_var_sklearn = pca_sklearn.explained_variance_ratio_

axes[1].bar(range(1, len(explained_var_custom) + 1), explained_var_custom, alpha=0.7, label='Custom', color='blue')
axes[1].bar(range(1, len(explained_var_sklearn) + 1), explained_var_sklearn, alpha=0.7, label='sklearn', color='red')
axes[1].set_xlabel('Componente Principal')
axes[1].set_ylabel('Varianza Explicada')
axes[1].set_title('Comparación de Varianza Explicada')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Análisis de varianza completado")


In [ ]:
print("\n" + "=" * 70)
print("PASO 10: REPORTE FINAL Y CONCLUSIONES")
print("=" * 70)

print("REPORTE DE VALIDACIÓN - IMPLEMENTACIÓN PCA DESDE CERO")
print("=" * 70)

print("DATOS")
print("-" * 70)
print(f"Dimensiones originales:  {X.shape[0]} muestras x {X.shape[1]} características")
print(f"Dimensiones reducidas:   {X_pca_custom.shape[0]} muestras x {X_pca_custom.shape[1]} componentes")
print(f"Reducción dimensional:   {(1 - X_pca_custom.shape[1]/X.shape[1])*100:.1f}%")
print(f"Número de clases:        {len(np.unique(y))}")

print("\nVARIANZA EXPLICADA")
print("-" * 70)
print(f"Implementación propia:")
for i, var in enumerate(explained_var_custom[:5]):
    print(f"  PC{i+1}: {var*100:.2f}%")
print(f"  Total: {np.sum(explained_var_custom)*100:.2f}%")

print(f"\nsklearn:")
for i, var in enumerate(explained_var_sklearn):
    print(f"  PC{i+1}: {var*100:.2f}%")
print(f"  Total: {np.sum(explained_var_sklearn)*100:.2f}%")

print("\nVALIDACIÓN")
print("-" * 70)
print(f"Correlación entre implementaciones:")
for i, corr in enumerate(correlations):
    print(f"  PC{i+1}: {corr:.6f}")

print(f"\nMétricas de error:")
print(f"  - MSE:           {mse:.6e}")
print(f"  - MAE:           {mae:.6e}")
print(f"  - Max diff:      {max_diff:.6e}")
print(f"  - Mean diff:     {mean_diff:.6e}")

print(f"\nInterpretación:")
print(f"  - Correlación > 0.99:  Excelente concordancia")
print(f"  - Correlación > 0.95:  Buena concordancia")
print(f"  - Correlación < 0.95:  Revisar implementación")

print(f"\nCONCLUSIÓN")
print("-" * 70)
print(f"Estado de la implementación: {status}")
print(f"Correlación promedio: {avg_corr:.6f}")

if avg_corr > 0.99:
    print(f"\n✓ La implementación produce resultados prácticamente idénticos a sklearn.")
    print(f"✓ El algoritmo PCA implementado desde cero es correcto y eficiente.")
elif avg_corr > 0.95:
    print(f"\n✓ La implementación produce resultados muy similares a sklearn.")
    print(f"✓ Pequeñas diferencias pueden deberse a precisiones numéricas.")
else:
    print(f"\n⚠ La implementación requiere revisión para mejorar la precisión.")

print(f"\nCARACTERÍSTICAS DE LA IMPLEMENTACIÓN")
print("-" * 70)
print(f"✓ Algoritmo Power Iteration para eigenvalores")
print(f"✓ Ordenamiento por eigenvalues descendente")
print(f"✓ Centrado de datos automático")
print(f"✓ Cálculo de matriz de covarianza")
print(f"✓ Proyección a componentes principales")
print(f"✓ Compatible con diferentes tipos de datos")
print(f"✓ Validación exhaustiva contra sklearn")

print(f"\n" + "=" * 70)
print("ANÁLISIS COMPLETO FINALIZADO EXITOSAMENTE")
print("=" * 70)
